# Setting up new parameters for an existing overlay

We'll now have a look at how to set up new parameters from a trained network for an existing overlay.
This notebook will be somewhat different than the rest of the BNN-PYNQ notebooks, as it will mostly consist of invocations of shell commands and existing Python scripts.

In this example, we will operate directly inside the existing BNN-PYNQ installation. This is not good practice since it will e.g. prevent a clean uninstallation, but here we do this for rapid development and convenience.

BNN-PYNQ comes with a folder with BinaryNet (using Lasagne + Theano) training scripts to reproduce the existing networks. We will start by having a look at what's inside.


In [ ]:
import bnn
root_dir = bnn.bnn.BNN_ROOT_DIR
param_dir = bnn.bnn.BNN_PARAM_DIR
training_dir = root_dir + "/src/training"
!ls $training_dir

In the rest of this notebook, we will set up a Fashion-MNIST network on the LFC topology. Here is a brief overview of the files that we will use for this:
* fashion-mnist-train/download_trained_params.sh to download the pre-trained Theano/BinaryNet parameters
* fashion-mnist-gen-binary-weights.py to convert the Theano/BinaryNet parameters to BNN-PYNQ parameters for the LFC overlay
* **If you'd like to train from scratch:** First use the fashion-mnist-train/download_data.sh script, then execute fashion-mnist.py but this would take quite some time to train. If you'd like to try this out yourself, you can read the README.md in the training directory for setting up Lasagne and Theano.
* **More details: ** lfc.py creates the network structure supported by the LFC overlay. finnthesizer.py contains the conversion logic for packing weights into the overlay memories, absorbing batchnorm into thresholds and so on.

## Getting the Trained Network Parameters

We will start by downloading the trained parameters using the provided script:

In [ ]:
!cd $training_dir/fashion-mnist-train; sh download_trained_params.sh

The file we just downloaded containts the trained the network parameters as a compressed numpy array. Before these can be used by the BNN-PYNQ overlay, they need to be converted and packed appropriately. This involves "folding" the weight matrices into the processing elements, as decribed by the FINN paper.

## Generating the BNN-PYNQ Weights

We can now use the **fashion-mnist-gen-binary-weights.py** script, which is very similar to the other gen-binary-weights scripts using LFC (just using a different parameter file), to produce the BNN-PYNQ weight files.

In [ ]:
!cd $training_dir; python fashion-mnist-gen-binary-weights.py

## Using the New Weights with the LFC Overlay

Once the weights have been generated, we can copy the new weight folder into the BNN-PYNQ parameter folder, such that they will be detected by the Python API. Afterwards, we can instantiate a classifier and load the weights to start classifying images.

In [ ]:
!cp -r $training_dir/fashion-mnist-lfc $param_dir/
bnn.available_params(bnn.NETWORK_LFC)

In [ ]:
c = bnn.PynqBNN(network=bnn.NETWORK_LFC)
c.load_parameters("fashion-mnist-lfc")
print(c.classes)

In [ ]:
from PIL import Image
# load image using PIL and convert to black and white
img_ankleboot = Image.open("ankle_boot.jpg").convert("L")
img_ankleboot

In [ ]:
with open("ankleboot-mnist-formatted", "wb") as fp:
  c.image_to_mnist(img_ankleboot, fp, invert=True)
ret_ankleboot = c.inference("ankleboot-mnist-formatted")
print("Predicted class: " + c.classes[ret_ankleboot])